In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from timeit import timeit
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from joblib import dump, load
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# df: pd.DataFrame = load("dumped_datasets/survival_all.joblib")
df: pd.DataFrame = load("dumped_datasets/road_all.joblib")
# df: pd.DataFrame = load("dumped_datasets/hisingen_all.joblib")

df.drop(columns=["data", "data_dec", "ID", "DLC", "t"], inplace=True, errors="ignore")
df.drop(columns=["d0", "d1", "d2", "d3", "d4", "d5", "d6", "d7"], inplace=True, errors="ignore")

# display(df)

In [ ]:
# Stratify on the sub-dataset
X_train = df.drop(columns="Label")
y_train = df["Label"]

df = None # Release memory

# Split dataset into training and test data, stratify by the type of attack
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=0, shuffle=True, stratify=X_train["name"])

In [ ]:
# rus = RandomUnderSampler(random_state=0)
# X_train, y_train = rus.fit_resample(X_train, y_train)
# X_test, y_test = rus.fit_resample(X_test, y_test)
# bintr = np.bincount(y_train)
# binte = np.bincount(y_test)
# print(f"Labels\t\tTraining\tTesting\nNormal\t\t{bintr[0]}\t\t{binte[0]}\nAttack\t\t{bintr[1]}\t\t{binte[1]}")

In [ ]:
name_train = X_train["name"]
# name_test = X_test["name"]
X_train.drop(columns=["type", "dataset", "name", "class"], inplace=True)
X_test.drop(columns=["type", "dataset", "name", "class"], inplace=True)

In [ ]:
from pp_tools import scale_dataset

X_train_original = X_train.copy()
X_test_original = X_test.copy()

X_combined = pd.concat([X_train, X_test])

scale_dataset(X_combined)

test_len = len(X_test)
X_test = X_combined.iloc[-test_len:]
X_train = X_combined.iloc[:-test_len]

X_combined = None # Release memory

vvv RF vvv

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=20, random_state=0, max_leaf_nodes=300, max_features="log2", warm_start=True)
rf.fit(X_train.values, y_train)

vvv DNN vvv

In [ ]:
import keras
dnn = keras.models.load_model("dnn_all_road_imbalanced")

vvv SHAP vvv

In [ ]:
import shap
import tensorflow as tf

X_sample = pd.concat([X_train[y_train == 1].sample(50, random_state=0), X_train[y_train == 0].sample(50, random_state=0)]).values

# vvv RF vvv

exp_rf = shap.KernelExplainer(rf.predict_proba, data=X_sample, algorithm="kernel")
exp_rf_tree = shap.TreeExplainer(rf)

# vvv DNN vvv

exp_dnn = shap.KernelExplainer(dnn.predict, data=X_sample, algorithm="kernel")
tf.compat.v1.disable_v2_behavior()
exp_dnn_deep = shap.DeepExplainer((dnn.layers[0].input, dnn.layers[-1].output), data=X_sample)
exp_dnn_grad = shap.GradientExplainer((dnn.layers[0].input, dnn.layers[-1].output), data=X_sample)

In [ ]:
from shap_tools import *

max_samples = int(min(len(X_train), 600) / 2)

X_exp = pd.concat([X_train[y_train == 1].sample(max_samples, random_state=0), X_train[y_train == 0].sample(max_samples, random_state=0)])
# X_exp = X_train.sample(min(len(X_train), 600), random_state=0)
y_exp = y_train[X_exp.index]
X_exp_original = X_train_original.loc[X_exp.index]

In [ ]:
def get_explanation2(explainer, df: pd.DataFrame):
    return shap.Explanation(
        values=explainer.shap_values(df.values)[-1],
        base_values=0.5, #explainer.expected_value[-1],
        data=df.to_numpy(),
        feature_names=df.columns)

In [ ]:
shap_rf = get_explanation2(exp_rf, X_exp)
shap_rf_tree = get_explanation2(exp_rf_tree, X_exp)

shap_dnn = get_explanation2(exp_dnn, X_exp)
shap_dnn_deep = get_explanation2(exp_dnn_deep, X_exp)
shap_dnn_grad = get_explanation2(exp_dnn_grad, X_exp)

In [ ]:
def plot_beeswarm2(exp_obj):
    cbar = plt.colorbar(plt.cm.ScalarMappable(cmap="plasma"), label="Feature value")
    cbar.set_ticks([0, 1])
    cbar.set_ticklabels(["Low", "High"])
    vis = shap.plots.beeswarm(exp_obj, max_display=20, color=plt.get_cmap("plasma"), color_bar=False)

In [ ]:
plot_beeswarm2(shap_rf)
plot_beeswarm2(shap_rf_tree)
plot_beeswarm2(shap_dnn)
plot_beeswarm2(shap_dnn_deep)
plot_beeswarm2(shap_dnn_grad)

In [ ]:
# shap.summary_plot(shap_rf, plot_type="bar")
# shap.summary_plot(shap_rf_tree, plot_type="bar")
# shap.summary_plot(shap_dnn, plot_type="bar")
# shap.summary_plot(shap_dnn_deep, plot_type="bar")
# shap.summary_plot(shap_dnn_grad, plot_type="bar")

tata = pd.DataFrame([shap_rf, shap_rf_tree, shap_dnn, shap_dnn_deep, shap_dnn_grad])

# sns.barplot()

In [ ]:
def euclid(a, b):
    return np.sqrt(np.dot(b-a, b-a))

def cosim(a, b):
    return np.dot(a, b)/(np.sqrt(np.dot(a, a)) * np.sqrt(np.dot(b, b)))

shap_rf_avg = np.array(shap_rf.values).flatten() # np.mean(shap_rf.values, axis=0)
shap_dnn_avg = np.array(shap_dnn.values).flatten() # np.mean(shap_dnn.values, axis=0)

# print(shap_rf_avg)
# print(shap_dnn_avg)

# shap_cosims = np.sort(list(map(cosim, shap_rf.values, shap_rf_tree.values)))
# shap_cosims = list(map(cosim, shap_rf.values, shap_rf_tree.values))

all_shaps = [shap_rf, shap_rf_tree, shap_dnn, shap_dnn_deep, shap_dnn_grad]
names = ["RF_KernelExplainer", "RF_TreeExplainer", "DNN_KernelExplainer", "DNN_DeepExplainer", "DNN_GradientExplainer"]

temp = pd.DataFrame(index=names, columns=names, dtype=np.float64)
for i in range(5):
    for j in range(5):
        temp.iloc[i, j] = np.mean(list(map(cosim, all_shaps[i].values, all_shaps[j].values)))

# Drop first row and last column that don't provide information
temp.drop(index=temp.index[0], inplace=True)
temp.drop(columns=temp.columns[-1], inplace=True)

# Generate a mask for the upper triangle but not the diagonal
mask = np.triu(np.ones_like(temp, dtype=bool))
np.fill_diagonal(mask, False)

sns.heatmap(temp, annot=True, square=True, mask=mask, vmin=0.7, vmax=1, center=0.7, annot_kws={"fontsize": 12}, fmt=".3f")
plt.xticks(rotation=20, horizontalalignment="right")

# print(np.mean(shap_cosims))
# print(euclid(shap_rf_avg, shap_dnn_avg))
# plt.plot(shap_cosims)